https://github.com/azure-ai-foundry/foundry-samples/tree/main/samples/microsoft/python/getting-started-agents/3p-tools/legalfly

In [ ]:
import os
import time
import jsonref
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import OpenApiTool, OpenApiConnectionAuthDetails, OpenApiConnectionSecurityScheme
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

In [ ]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# Connection to the legalfly API (Azure AI Foundry project custom key connection)
connection_name = os.environ["LEGALFLY_API_CONNECTION_NAME"]

In [18]:
# Load the OpenAPI specification for the service from a local JSON file using jsonref to handle references
with open("./legalfly.json", "r") as f:
    openapi_spec = jsonref.loads(f.read())

conn_id = project_client.connections.get(name=connection_name).id
# Create Auth object for the OpenApiTool (note that connection or managed identity auth setup requires additional setup in Azure)
auth = OpenApiConnectionAuthDetails(security_scheme=OpenApiConnectionSecurityScheme(connection_id=conn_id))

# Initialize the main OpenAPI tool definition for weather
openapi_tool = OpenApiTool(
    name="getLegalCounsel", 
    spec=openapi_spec, 
    description="LegalFly legal counsel API", 
    auth=auth
)

In [ ]:
# Create an agent configured with the combined OpenAPI tool definitions
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"], # Specify the model deployment
    name="my-legal-agent", # Give the agent a name
    instructions="You are a helpful AI legal assistant. Act like a friendly person who possesses a lot of legal knowledge.",
    tools=openapi_tool.definitions, # Provide the list of tool definitions
)
print(f"Created agent, ID: {agent.id}")

In [ ]:
# Create a new conversation thread for the interaction
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Create the initial user message in the thread
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    # give an example of a user message that the agent can respond to
    content="What do I need to start a company in California?",
)
print(f"Created message, ID: {message.id}")

In [ ]:
run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

while run.status in ("queued", "in_progress"):
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

print(f"Run finished with status: {run.status}, ID: {run.id}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Delete the agent resource to clean up
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

In [22]:
# Fetch and log all messages exchanged during the conversation thread
messages = project_client.agents.messages.list(thread_id=thread.id)
for m in messages:
    content = m.get("content", [])
    if content and content[0].get("type") == "text":
        text_value = content[0].get("text", {}).get("value", "")
        print(f"Text: {text_value}")

Text: Starting a company in California is an exciting endeavor! Here’s a practical checklist of what you’ll typically need:

---

**1. Decide on Your Business Structure**
- **Sole Proprietorship, Partnership, LLC, Corporation, or others** (LLC and Corporation are the most common for liability protection).

**2. Choose and Reserve a Business Name**
- Check availability using the [California Secretary of State’s website](https://businesssearch.sos.ca.gov/).
- Make sure you comply with naming rules for your chosen entity type.
- Optionally reserve your name online or by mail.

**3. Register Your Business**
- **LLC:** File Articles of Organization (Form LLC-1) with the Secretary of State.
- **Corporation:** File Articles of Incorporation (Form ARTS-GS for general stock).
- **Sole Proprietorship/Partnership:** Generally, no state-level filing required.

**4. Obtain an Employer Identification Number (EIN)**
- Apply for free at the [IRS website](https://www.irs.gov/businesses/small-businesses